ANPR-Traffic-Plate-Recognition

In [ ]:
%%capture
!pip install ultralytics easyocr opencv-python roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="Ft4uvDzqztrfLBv5grnn")  # Get your free key in 30 seconds at https://app.roboflow.com
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(4)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Recognition-4 in yolov8:: 100%|██████████| 48488/48488 [00:06<00:00, 7039.20it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=30 imgsz=640 batch=16 plots=True

Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/License-Plate-Recognition-4/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

In [ ]:
# This will show you exactly where your model was saved
!ls -la /content/runs/detect/train/weights/

In [ ]:
from ultralytics import YOLO

# Automatically find the latest training folder
import glob
import os

# Find the latest 'train' folder
train_dirs = sorted(glob.glob("/content/runs/detect/train*"))
latest_train = train_dirs[-1] if train_dirs else "/content/runs/detect/train"

model_path = os.path.join(latest_train, "weights/best.pt")

# Load the model
model = YOLO(model_path)
print(f"Model loaded successfully from: {model_path}")


In [ ]:
!ls /content/

In [ ]:
import os
dataset_path = "/content/License-Plate-Recognition-4"   # ← CHANGE THIS to whatever you saw above

test_images_path = os.path.join(dataset_path, "test/images")
print("Looking for test images here:", test_images_path)
!ls {test_images_path} | head -5   # This should show some .jpg files

In [ ]:
# Replace with the correct folder you found
correct_folder = "License-Plate-Recognition-4"   # ← PUT YOUR ACTUAL FOLDER NAME HERE

# Test prediction (this will work now)
results = model.predict(
    source=f"/content/{correct_folder}/test/images",
    save=True,
    conf=0.5,
    project="/content/results"
)

print("Done! Check the folder /content/results/predict for output images with detected plates")

In [ ]:
# Auto-detect the real dataset folder and run prediction
import glob, os
dataset_folder = glob.glob("/content/*Plate*")[0]  # finds any folder with "Plate" in name
model.predict(source=f"{dataset_folder}/test/images", save=True, conf=0.5)
!echo "Check results here:" && ls /content/runs/detect/predict*

In [ ]:
import cv2
import easyocr
from google.colab.patches import cv2_imshow
import glob

# Initialize OCR reader (only once)
reader = easyocr.Reader(['en'], gpu=True)  # Add 'hi' for Hindi/Indian plates if needed

# Auto-find your dataset folder
dataset_folder = glob.glob("/content/*Plate*")[0]
test_images = glob.glob(f"{dataset_folder}/test/images/*.jpg")[:10]  # First 10 images

for img_path in test_images:
    img = cv2.imread(img_path)
    results = model(img, conf=0.5)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        confidence = box.conf[0]

        # Crop the plate
        plate_crop = img[y1:y2, x1:x2]

        # OCR
        ocr_result = reader.readtext(plate_crop, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ-', detail=0)
        plate_text = ocr_result[0] if ocr_result else "??????"

        # Draw everything
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(img, f"{plate_text} ({confidence:.2f})", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2_imshow(img)
    print(f"Detected: {plate_text}\n")

In [ ]:
# List files to confirm
!ls -lh "Traffic Control CCTV.mp4"

# Move/rename it so we can use a clean name (optional but recommended)
!cp "Traffic Control CCTV.mp4" traffic_input.mp4

In [ ]:
import cv2
import easyocr
from google.colab.patches import cv2_imshow

# Initialize OCR (only once)
reader = easyocr.Reader(['en'], gpu=True)   # change to ['hi','en'] if Indian plates

# Open your uploaded video
cap = cv2.VideoCapture("traffic_input.mp4")

# Get video properties
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video loaded: {width}×{height}, {fps:.2f} FPS, {total_frames} frames")

# Output video
out = cv2.VideoWriter('MY_ANPR_RESULT.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps, (width, height))

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO detection
    results = model(frame, conf=0.35, iou=0.5, verbose=False)[0]  # lowered conf a bit for real CCTV

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf.item()

        # Crop plate region
        plate = frame[y1:y2, x1:x2]
        if plate.size == 0:
            continue

        # OCR (tuned for real-world plates)
        text_list = reader.readtext(plate,
                                    allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ- ',
                                    detail=0,
                                    paragraph=False,
                                    width_ths=0.8,
                                    height_ths=0.8)
        plate_text = text_list[0].upper() if text_list else "???"

        # Draw results
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(frame, plate_text, (x1, y1-15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 3)
        cv2.putText(frame, f"{conf:.2f}", (x1, y2+30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    out.write(frame)
    frame_idx += 1
    if frame_idx % 50 == 0 or frame_idx == total_frames:
        print(f"Processed {frame_idx}/{total_frames} frames...")

cap.release()
out.release()
print("FINISHED! Your result video → MY_ANPR_RESULT.mp4")

In [ ]:
from google.colab import files
files.download('MY_ANPR_RESULT.mp4')